In [9]:
from configparser import ConfigParser
from datetime import datetime
import pandas as pd
import pyodbc

In [10]:
# CHANGE THE DIRECTORY TO NOTEBOOK/NOTEBOOK
try:
    cd /notebooks/notebooks
except:
    %cd /notebooks/notebooks

/notebooks/notebooks


In [11]:
CurrentDate = str(datetime.now())
delimiter1 = " ";
delimiter2 = "-";
CurrentDate = CurrentDate.replace(delimiter2, delimiter1);
CD    = CurrentDate.split(delimiter1);
date  = CD[2]
month = CD[1]
year  = CD[0]

In [12]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [13]:
# CHECK IF JOB IS RUNNING FOR THE FIRST TIME.
count_of_audit_ingt = pd.read_sql_query('''SELECT COUNT(1) FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN (NOLOCK) 
                                           WHERE JOB_NAME = 'TRANSACTION_RISK_PROCESSING' AND JOB_STATUS = 'SUCCESS' ''',conn)

In [14]:
count_of_audit_ingt = count_of_audit_ingt.iloc[0,0]

0

In [15]:
def update_peerGroup():
        cursor.execute('''UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG  SET PEER_GROUP_ID=T.PEER_GROUP_ID 
                          FROM (SELECT DISTINCT A.ENTITY_ID,B.PEER_GROUP_ID,A.ENTITY_TYPE 
                          FROM [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG A WITH (NOLOCK)
                          INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID = A.REPORT_ID
                          INNER JOIN  FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG B WITH (NOLOCK) ON A.ENTITY_ID=B.PERSON_MASTER_ID WHERE A.ENTITY_TYPE ='I'  
                          UNION
                          SELECT DISTINCT A.ENTITY_ID,B.PEER_GROUP_ID,A.ENTITY_TYPE 
                          FROM [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG A WITH (NOLOCK)
                          INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK)
                          ON RTI.REPORT_ID = A.REPORT_ID
                          INNER JOIN  FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG B WITH (NOLOCK)
                          ON A.ENTITY_ID=B.ORGANIZATION_MASTER_ID WHERE A.ENTITY_TYPE ='O'  )T
                          WHERE T.ENTITY_ID=FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG.ENTITY_ID
                          AND T.ENTITY_TYPE=FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG.ENTITY_TYPE''')
        print('pass')  
        conn.commit()
        rules_df = pd.read_sql_query("SELECT * FROM FINCORE_ANALYTICS.RISK_LKP_RULES WITH (NOLOCK) WHERE RULE_NO LIKE 'T%'  AND IS_ACTIVE_FLAG = 1 AND Query IS NOT NULL",conn)
        #Populate the risk_person_summary_stg        
        for query in rules_df['Query'].to_list():
            print(query)
            cursor.execute(query)
            conn.commit()
        cursor.execute('''UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES  
                          SET RULE_VALUE_CONSOLIDATED=T.RULE_VALUE_CONSOLIDATED 
                          FROM (SELECT ID_,
                          SUM(CAST(RULE_VALUE AS DECIMAL(20,2))) OVER (PARTITION BY A.REPORT_ID,A.RULE_NO) RULE_VALUE_CONSOLIDATED,
                          A.RULE_NO,A.REPORT_ID
                          FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES A (NOLOCK)
                          INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK)
                          ON RTI.REPORT_ID = A.REPORT_ID
                          INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES B (NOLOCK) ON A.RULE_NO = B.RULE_NO
                          WHERE ISNUMERIC(RULE_VALUE) = 1 AND B.RULE_DESC NOT LIKE '%FREQUENCY%')T
                          WHERE T.ID_=FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.ID_ 
                          AND T.RULE_NO =  FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.RULE_NO
                          AND T.REPORT_ID =  FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.REPORT_ID''')
        print('PASS')
        conn.commit()


In [16]:
try:
    no_of_records = pd.read_sql_query('''SELECT COUNT(1) FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL (NOLOCK)''' ,conn)
    no_of_records = no_of_records.iloc[0,0]
    %store no_of_records
    if ((date == '01' and month == '01') or (date == '01' and month == '06') or (count_of_audit_ingt == 0)):        
        
        update_peerGroup()
        cursor.execute("""INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,
                          TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE) VALUES
                          (COALESCE((SELECT (MAX(JOB_RUN_ID)+1) FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN),1), 
                          'TRANSACTION_RISK_PROCESSING',GETDATE(),'ERROR','RISK_TRANSACTION_SUMMARY_STG','CPS_REPORT_DETAILS',
                          (SELECT MAX(UPDATE_DATE) FROM FINCORE_DB.FINCORE.CPS_REPORT_DETAILS))""")
        conn.commit()
        job_id = pd.read_sql_query('''SELECT MAX(JOB_RUN_ID) AS JOB_RUN_ID FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN (NOLOCK) 
                                      WHERE JOB_NAME = 'TRANSACTION_RISK_PROCESSING' ''',conn)
        job_id = job_id.iloc[0,0]
        %store job_id
        %run Transaction_Risk/RISK_LKP_ENTITY_THRESHOLD.ipynb
        %run Transaction_Risk/Transaction_risk_final_notebook.ipynb 
        config = ConfigParser()
        config.read(r'/notebooks/notebooks/config.ini')
        connection_string = config.get('SERVER', 'connection')
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        cursor.execute(f'''UPDATE FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN SET JOB_END_DATE = GETDATE(),JOB_STATUS = 'SUCCESS',
                           NO_OF_RECORD = {no_of_records},SOURCE_MAX_DATE = (SELECT MAX(CREATE_DATE) 
                           FROM FINCORE_DB.FINCORE.CPS_REPORT_DETAILS WITH (NOLOCK)) WHERE  JOB_RUN_ID ={job_id} 
                           AND JOB_NAME='TRANSACTION_RISK_PROCESSING' ''')
        conn.commit()
    else:
        print("Else")
        update_peerGroup()
        cursor.execute("""INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,
                          TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)VALUES
                          (COALESCE((SELECT (MAX(JOB_RUN_ID)+1) FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN),1), 
                          'TRANSACTION_RISK_PROCESSING',GETDATE(),'ERROR','RISK_TRANSACTION_SUMMARY_STG','CPS_REPORT_DETAILS',
                          (SELECT MAX(UPDATE_DATE) FROM FINCORE_DB.FINCORE.CPS_REPORT_DETAILS))""")
        conn.commit()
        job_id = pd.read_sql_query('''SELECT MAX(JOB_RUN_ID) AS JOB_RUN_ID FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN (NOLOCK) 
                                      WHERE JOB_NAME = 'TRANSACTION_RISK_PROCESSING' ''',conn)
        job_id = job_id.iloc[0,0]
        %store job_id
        %run Transaction_Risk/Transaction_risk_final_notebook.ipynb
        config.read(r'/notebooks/notebooks/config.ini')
        connection_string = config.get('SERVER', 'connection')
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        cursor.execute(f'''UPDATE FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN SET JOB_END_DATE = GETDATE(),JOB_STATUS = 'SUCCESS',
                           NO_OF_RECORD = {no_of_records} ,SOURCE_MAX_DATE =  (SELECT MAX(CREATE_DATE) 
                           FROM FINCORE_DB.FINCORE.CPS_REPORT_DETAILS WITH (NOLOCK)) WHERE JOB_RUN_ID ={job_id} 
                           AND JOB_NAME='TRANSACTION_RISK_PROCESSING' ''')
        conn.commit()
        
       
except Exception as error:
    %store -r job_id
    error = str(error)
    print("Here error")
    error = error.replace('\'','')
    error = error.replace('\"','')
    print("Failed to update record to database rollback: {}".format(error))
    conn.rollback()
    conn.execute(f'''UPDATE FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  
                     JOB_RUN_ID = {job_id}''')
    conn.commit()
    conn.close()

Stored 'no_of_records' (int64)
pass
MERGE INTO [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_RULES AS TARGET USING ( SELECT DISTINCT A.BATCH_ID, A.REPORT_ID, A.ENTITY_ID,'T003003001' AS RULE_NO ,A.TRAN_TYP AS RULE_VALUE,A.PEER_GROUP_ID,B.RISK_SCORE AS RISK_SCORE_STATIC,GETDATE() AS CREATE_DATE,A.ENTITY_TYPE FROM [FINCORE_DB].FINCORE_ANALYTICS.[RISK_TRANSACTION_SUMMARY_STG] A WITH (NOLOCK) INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  A.REPORT_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_SCORE_RNG B WITH(NOLOCK) ON  B.ATTRIBUTE_SUBTYP=A.TRAN_TYP WHERE B.ATTRIBUTE_TYP='TRN_TYP' GROUP BY A.BATCH_ID,A.REPORT_ID,A.ENTITY_ID, A.ENTITY_TYPE,A.TRAN_TYP,B.RISK_SCORE,A.PEER_GROUP_ID) AS SOURCE ON  TARGET.BATCH_ID = SOURCE.BATCH_ID AND TARGET.REPORT_ID = SOURCE.REPORT_ID AND TARGET.ENTITY_ID = SOURCE.ENTITY_ID AND TARGET.RULE_NO =SOURCE.RULE_NO AND TARGET.ENTITY_TYPE = SOURCE.ENTITY_TYPE AND TARGET.RULE_VALUE = SOURCE.RULE_VALUE W

MERGE INTO [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_RULES AS TARGET USING ( SELECT A.ID_, BATCH_ID, A.REPORT_ID, A.ENTITY_ID, 'T003003022' AS RULE_NO, FP.PINCODENUMBER AS RULE_VALUE, A.PEER_GROUP_ID, AVG(FP.RISK_SCORE) OVER (PARTITION BY A.ENTITY_ID) AS RISK_SCORE_STATIC, GETDATE() AS CREATE_DATE, A.ENTITY_TYPE FROM [FINCORE_DB].FINCORE_ANALYTICS.[RISK_TRANSACTION_SUMMARY_STG] A WITH (NOLOCK) INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  A.REPORT_ID INNER JOIN DATAHUB_DB.FIUMETAHUB.FINNET_PINCODE FP ON A.LOCATION_PINCODE =FP.PINCODENUMBER WHERE A.REPORT_TYPE='NTR') AS SOURCE ON TARGET.BATCH_ID = SOURCE.BATCH_ID AND TARGET.ID_ = SOURCE.ID_ AND TARGET.REPORT_ID = SOURCE.REPORT_ID AND TARGET.ENTITY_ID = SOURCE.ENTITY_ID AND TARGET.RULE_NO =SOURCE.RULE_NO AND TARGET.ENTITY_TYPE = SOURCE.ENTITY_TYPE AND TARGET.RULE_VALUE = SOURCE.RULE_VALUE WHEN MATCHED THEN UPDATE SET TARGET.RULE_VALUE = SOURCE.RULE_VALUE, TARGET.RISK_SCORE_S

MERGE INTO [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_RULES AS TARGET USING ( SELECT ID_, BATCH_ID,  A.REPORT_ID, ENTITY_ID, 'T003003028_D' AS RULE_NO,COUNT(1) OVER (PARTITION BY A.REPORT_ID,ENTITY_ID,ENTITY_TYPE,A.PEER_GROUP_ID) AS RULE_VALUE, A.PEER_GROUP_ID,0 AS RISK_SCORE_STATIC, GETDATE() AS CREATE_DATE, A.ENTITY_TYPE FROM [FINCORE_DB].FINCORE_ANALYTICS.[RISK_TRANSACTION_SUMMARY_STG] A (NOLOCK) INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  A.REPORT_ID WHERE A.TRAN_TYP='TS1' AND A.CR_DR_DEP_WITH='CREDIT' ) AS SOURCE ON TARGET.BATCH_ID = SOURCE.BATCH_ID AND TARGET.ID_ = SOURCE.ID_ AND TARGET.REPORT_ID = SOURCE.REPORT_ID AND TARGET.ENTITY_ID = SOURCE.ENTITY_ID AND TARGET.RULE_NO =SOURCE.RULE_NO AND TARGET.ENTITY_TYPE = SOURCE.ENTITY_TYPE WHEN MATCHED THEN UPDATE SET TARGET.RULE_VALUE = SOURCE.RULE_VALUE, TARGET.RISK_SCORE_STATIC=SOURCE.RISK_SCORE_STATIC, TARGET.CREATE_DATE = SOURCE.CREATE_DATE WHEN NOT MATCHED BY TARGET T

MERGE INTO [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_RULES AS TARGET USING ( SELECT A.ID_, BATCH_ID, A.REPORT_ID, A.ENTITY_ID, A.ENTITY_TYPE, 'T003003036' AS RULE_NO, COUNT(A.TRANSACTION_ID) OVER (PARTITION BY A.REPORT_ID, A.ENTITY_ID,A.ENTITY_TYPE,A.PEER_GROUP_ID ) AS RULE_VALUE, A.PEER_GROUP_ID, 0 AS RISK_SCORE_STATIC, GETDATE() AS CREATE_DATE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG A WITH (NOLOCK) INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  A.REPORT_ID WHERE A.ENTITY_TYPE IS NOT NULL AND A.REPORT_TYPE='NTR' AND A.CR_DR_DEP_WITH IN ('CREDIT') ) AS SOURCE ON TARGET.BATCH_ID = SOURCE.BATCH_ID AND TARGET.ID_ = SOURCE.ID_ AND TARGET.REPORT_ID = SOURCE.REPORT_ID AND TARGET.ENTITY_ID = SOURCE.ENTITY_ID AND TARGET.RULE_NO ='T003003036' AND TARGET.ENTITY_TYPE = SOURCE.ENTITY_TYPE WHEN MATCHED THEN UPDATE SET TARGET.RULE_VALUE = SOURCE.RULE_VALUE, TARGET.RISK_SCORE_STATIC=SOURCE.RISK_SCORE_STATIC, TARGET.

MERGE INTO [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_RULES AS TARGET USING ( SELECT RTSS.ID_,RTSS.BATCH_ID, RTSS.REPORT_ID, RTSS.ENTITY_ID, RTSS.ENTITY_TYPE, 'T003003041' AS RULE_NO, COUNT( RTSS.TRANSACTION_ID) OVER(PARTITION BY RTSS.REPORT_ID, RTSS.ENTITY_ID,RTSS.ENTITY_TYPE,RTSS.PEER_GROUP_ID) AS RULE_VALUE, RTSS.PEER_GROUP_ID, 0 AS RISK_SCORE_STATIC, GETDATE() AS CREATE_DATE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG RTSS WITH (NOLOCK) INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  RTSS.REPORT_ID INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS CRD WITH (NOLOCK) ON RTSS.BATCH_ID=CRD.FIU_BATCH_ID AND RTSS.REPORT_ID=CRD.FIU_REPORT_ID INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_CBWTRBANK CBWTR WITH (NOLOCK) ON CBWTR.BATCHID=CRD.BATCH_ID AND CBWTR.REPORTID=CRD.REPORT_ID INNER JOIN [DATAHUB_DB].[FIUMETAHUB].[FINNET_COUNTRY] FC ON CBWTR.DESTINATIONCOUNTRY=FC.ID_ WHERE RTSS.REPORT_TYPE='CBWTR' AND FC

...

1522001.0 1522002.0 1522003.0 1522004.0 1522005.0 1522006.0 1522007.0 1522008.0 1522009.0 1522010.0 1522000.0 1522000.0 1522000 0.0 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
Not applicable
thresh
13853813 17707626 21561439 25415252 29269065 33122878 36976691 40830504 44684317 48538130 7775000.0 10000000.0 10000000 3853813.046840752 NA
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.5 1.5 1 0.5 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 0 0.816496580927726 NA
Not applicable
Not applicable
thresh
1500000 2210000 2920000 3630000 4340000 5050000 5760000 6470000 7180000 7890000 790000.0 790000.0 80000 710000.0 NA
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not 

/tmp/pip_packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


0.6793662204867574 3.0
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
29351603 48653206 67954809 87256412 106558015 125859617 145161220 164462823 183764426 203066029 17775000.0 10050000.0 1000000 19301602.91271168 NA
Not applicable
thresh
29351603 48653206 67954809 87256412 106558015 125859617 145161220 164462823 183764426 203066029 17775000.0 10050000.0 1000000 19301602.91271168 NA
Not applicable
thresh
29351603 48653206 67954809 87256412 106558015 125859617 145161220 164462823 183764426 203066029 17775000.0 10050000.0 1000000 19301602.91271168 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
not_normal_outliers_removal
compute_IQR
outliers_right 4.0
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.3157894736842106 1.0 1 0.6115236862432764 4.0
normal_outliers_removal
1.0
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0

1000002.0 1000003.0 1000004.0 1000005.0 1000006.0 1000007.0 1000008.0 1000009.0 1000010.0 1000011.0 1000001.0 1000001.0 1000001 0.0 NA
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
Not applicable
Not applicable
thresh
497835 945670 1393504 1841339 2289174 2737009 3184843 3632678 4080513 4528348 366667.0 50000.0 50000 447834.7661560009 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
1000002.0 1000003.0 1000004.0 1000005.0 1000006.0 1000007.0 1000008.0 1000009.0 1000010.0 1000011.0 1000001.0 1000001.0 1000001 0.0 NA
Not applicable
Not applicable


Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
Not applicable
Not applicable
thresh
821210 992419 1163629 1334839 1506048 1677258 1848468 2019677 2190887 2362097 745200.0 650000.0 600000